In [ ]:
# load libraries
import defs
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time as t

In [ ]:
# set plots
%matplotlib inline
 
plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

In [ ]:
# get tickers
tickers = []

#get file object
f = open("list", "r")

while(True):
    #read next line
    line = f.readline()
    #if line is empty, you are done with all lines in the file
    if not line:
        break
    #you can access the line
    tickers.append(line.strip())

#close file
f.close()

In [ ]:
# calculates
manager = mp.Manager()
market_dfset=manager.dict()
modelset=manager.dict()
forecastset=manager.dict()

p = {}
for ticker in tickers:
    p[ticker]= mp.Process(target=defs.run_neuralprophet,args=(ticker,market_dfset,modelset,forecastset))
    
l = len(tickers)
c = mp.cpu_count()
for i in range(0, l, c):
    for j in range(0,c):
        if (i+j<l):
            p[tickers[i+j]].start()
    for j in range(0,c):
        if (i+j<l): p[tickers[i+j]].join()
            
print(market_dfset)

In [ ]:
# show graphs
figureset = {}
legendset = {}
for ticker in tickers:
    figureset[ticker] = market_dfset[ticker]["Close"].plot()
    legendset[ticker]=figureset[ticker].legend() #get the legend
    legendset[ticker].get_texts()[0].set_text(ticker) #change the legend text
    plt.show()
    
for ticker in tickers:
    n=3650
    print(ticker)
    print(forecastset[ticker])
    figure=modelset[ticker].plot(forecastset[ticker])  
    fig_param = modelset[ticker].plot_parameters()
    plt.show()
    print('\n')

In [ ]:
# merge the sets
full_dfset = {}
for ticker in tickers:
    full_dfset[ticker] = forecastset[ticker].set_index('ds').join(market_dfset[ticker])
    print(full_dfset[ticker])

In [ ]:
# plot the future
for ticker in tickers:
    fig, ax1 = plt.subplots()
    n=3 + 7
    d=7 + 3
    modelset[ticker].plot(forecastset[ticker])
    #ax1.plot(modelset[ticker][modelset[ticker].last_valid_index()-pd.DateOffset(n, 'D'):modelset[ticker].last_valid_index()-pd.DateOffset(n-d, 'D')], color='grey', linestyle=':')
    #ax1.plot(modelset[ticker][modelset[ticker].last_valid_index()-pd.DateOffset(n, 'D'):modelset[ticker].last_valid_index()-pd.DateOffset(n-d, 'D')].Close, color='red', linestyle='-')
#    ax1.plot(full_dfset[ticker].tail(n).yhat183, color='black', linestyle='--')
#    ax1.plot(full_dfset[ticker].tail(n).yhat365, color='black', linestyle='-.')
#    ax1.fill_between(full_dfset[ticker].tail(n).index, full_dfset[ticker]['yhat2'].tail(n), full_dfset[ticker]['yhat3'].tail(n), alpha=0.5, color='darkgray')
    ax1.set_title(ticker)
    ax1.set_ylabel('Price')
    ax1.set_xlabel('Date')
    plt.show()

In [ ]:
# write out
for ticker in tickers:
    n = 365
    full_dfset[ticker].tail(n).to_csv(ticker+'-'+str(n)+'-nprophet-Days.csv')